# 4. Spatio-temporal diversity of must 

In [1]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries

In [2]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries


In [3]:
%env TMPDIR=/scratch/lfloerl/tmpdata

env: TMPDIR=/scratch/lfloerl/tmpdata


In [4]:
import qiime2 as q2
from qiime2 import Visualization
from qiime2 import Artifact
from qiime2 import Metadata
from qiime2.plugins.diversity.visualizers import alpha_group_significance
from qiime2.plugins import diversity as q2d


import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import spearmanr


%matplotlib inline

In [5]:
md = Metadata.load('/home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv')
taxonomy = q2.Artifact.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/taxonomy.qza')

# Diversity 

In [6]:
table = q2.Artifact.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_table.qza')

* 10'000: Retained 3’400’000 (26.33%) features in 340 (95.51%) samples

* **15'000**: Retained 4’845’000 (37.51%) features in 323 (90.73%) samples

In [7]:
Visualization.load('/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_table.qzv')

<visualization: Visualization uuid: 3a3480e1-3ded-4aeb-9b61-6ce3afc75f11>

In [8]:
depth = 10000 
threads = 5

In [9]:
core_div_dict = {}
core_div = q2d.actions.core_metrics(table=table,
                                    sampling_depth=depth,
                                    metadata=md, 
                                    n_jobs=threads)
# save outputs
output_dir='must_cm{0}/'.format(depth)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for k, v in core_div.__dict__.items():
    if k != '_fields':
        v.save(output_dir + k)
        core_div_dict[k] = v

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/skbio/util/_warning.py:60: UserWarning: observed_otus is deprecated as of 0.6.0.
  warn(f"{func.__name__} is deprecated as of {ver}.")
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.4922440231911884 and the largest is 10.101415922711556.
  warn(


In [10]:
Visualization.load(f'{workdir}/{output_dir}/bray_curtis_emperor.qzv')

<visualization: Visualization uuid: b2d874c4-5f55-4711-a672-723dbf2d1861>

In [11]:
Visualization.load(f'{workdir}/{output_dir}/jaccard_emperor.qzv')

<visualization: Visualization uuid: f23a0465-4c0c-4e3b-a30a-d7b85cd6867d>

## Kmerizer 

In [29]:
#!rm -r kmer-core-metrics

In [92]:
%%bash

qiime feature-table filter-seqs \
    --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_table.qza \
    --o-filtered-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_rep_seqs.qza

qiime kmerizer core-metrics \
    --i-sequences /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_rep_seqs.qza \
    --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_table.qza \
    --p-sampling-depth 10000 \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
    --p-n-jobs 'auto' \
    --output-dir must_kmer_cm10000/

Saved FeatureData[Sequence] to: /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_rep_seqs.qza
Saved FeatureTable[Frequency] to: must_kmer_cm10000/rarefied_table.qza
Saved FeatureTable[Frequency] to: must_kmer_cm10000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: must_kmer_cm10000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: must_kmer_cm10000/shannon_vector.qza
Saved DistanceMatrix to: must_kmer_cm10000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: must_kmer_cm10000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: must_kmer_cm10000/jaccard_pcoa_results.qza
Saved PCoAResults to: must_kmer_cm10000/bray_curtis_pcoa_results.qza
Saved Visualization to: must_kmer_cm10000/scatterplot.qzv


In [12]:
Visualization.load('must_kmer_cm10000/scatterplot.qzv')

<visualization: Visualization uuid: 99a278e0-ba4c-4122-8647-5b83bd81dd1f>

### Alpha Group Significance 

In [22]:
# metrics: 
alpha_metrics = ['shannon_vector', 'observed_features_vector', 'evenness_vector']

for m in alpha_metrics:
    ags, = alpha_group_significance(alpha_diversity = core_div_dict[m],
                               metadata = md)
    ags.save(output_dir + 'alpha_groupSig_{0}.qzv'.format(m))

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[cols] = df[cols].apply(pd.to_numeric, errors='ignore')
/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/q2_types/sample_data/_deferred_setup/_transformers.py:28: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions

In [23]:
Visualization.load(f'{output_dir}/alpha_groupSig_evenness_vector.qzv')

<visualization: Visualization uuid: 4a90a9e7-e822-4199-86bc-667575f25597>

In [24]:
Visualization.load(f'{output_dir}/alpha_groupSig_observed_features_vector.qzv')

<visualization: Visualization uuid: e9b699af-3089-41a9-9c46-a831182757b7>

In [25]:
Visualization.load(f'{output_dir}/alpha_groupSig_shannon_vector.qzv')

<visualization: Visualization uuid: c45b7be7-6a7d-4c9e-8499-964acfb01e51>

### PERMANOVA

In [94]:
formula = 'Plot_ID + Year + Time_point + Plot_ID:Year + Plot_ID:Time_point + Year:Time_point'

beta_metrics = ['bray_curtis', 'jaccard']

# PERMANOVA with core metrics 
for m in beta_metrics:
    adonis_res, = q2d.actions.adonis(
        distance_matrix=core_div_dict[m + '_distance_matrix'],
        metadata=md,
        formula=formula,
        n_jobs=3)
    adonis_res.save(output_dir + 'adonis_{0}.qzv'.format(m))
    

# PERMANOVA with kmer core metrics 
kmer_dir = 'must_kmer_cm10000'
    
for m in beta_metrics:
    !qiime diversity adonis \
        --i-distance-matrix "{kmer_dir}/{m}_distance_matrix.qza" \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
        --p-formula "{formula}" \
        --p-n-jobs 5 \
        --o-visualization "{kmer_dir}/adonis_{m}.qzv"

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /scratch/lfloerl/tmpdata/tmpl7vjr_d1/dm.tsv /scratch/lfloerl/tmpdata/tmpl7vjr_d1/md.tsv Plot_ID + Year + Time_point + Plot_ID:Year + Plot_ID:Time_point + Year:Time_point 999 3 /scratch/lfloerl/tmpdata/qiime2-temp-crpchujp/adonis.tsv

R version 4.3.3 (2024-02-29) 


Lade nötiges Paket: permute
Lade nötiges Paket: lattice
This is vegan 2.6-8
Warnmeldung:
'adonis' ist veraltet.
Benutzen Sie stattdessen 'adonis2'
Siehe help("Deprecated") und help("vegan-deprecated"). 


Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: run_adonis.R /scratch/lfloerl/tmpdata/tmp6_2a70wr/dm.tsv /scratch/lfloerl/tmpdata/tmp6_2a70wr/md.tsv Plot_ID + Year + Time_point + Plot_ID:Year + Plot_ID:Time_point + Year:Time_point 999 3 /scratch/lfloerl/tmpdata/qiime2-temp-2gx_1mvp/adonis.tsv

R version 4.3.3 (2024-02-29) 


Lade nötiges Paket: permute
Lade nötiges Paket: lattice
This is vegan 2.6-8
Warnmeldung:
'adonis' ist veraltet.
Benutzen Sie stattdessen 'adonis2'
Siehe help("Deprecated") und help("vegan-deprecated"). 


Saved Visualization to: must_kmer_cm10000/adonis_bray_curtis.qzv
Saved Visualization to: must_kmer_cm10000/adonis_jaccard.qzv


In [95]:
Visualization.load(f'{output_dir}/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: 60a20862-78d4-4602-8709-4e722bb2287c>

In [96]:
Visualization.load(f'{output_dir}/adonis_jaccard.qzv')

<visualization: Visualization uuid: d595c452-e20a-4453-9f5f-763ec5f2cc0b>

In [97]:
Visualization.load(f'{kmer_dir}/adonis_bray_curtis.qzv')

<visualization: Visualization uuid: 8066ea2c-c9fc-4fc6-81c3-b114d0f58667>

In [98]:
Visualization.load(f'{kmer_dir}/adonis_jaccard.qzv')

<visualization: Visualization uuid: 927e6eef-64e8-4014-9a7f-a190563a39ca>

## Mantel 

In [100]:
%%bash 

div_dir='must_cm10000'
kmer_dir='must_kmer_cm10000'

# geodisic distance
#qiime tools export --input-path /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza --output-path geodesic_distance_ITS

# distance matrices 
qiime tools export --input-path $div_dir/bray_curtis_distance_matrix.qza --output-path $div_dir/bray_curtis_distance_matrix
qiime tools export --input-path $div_dir/jaccard_distance_matrix.qza --output-path $div_dir/jaccard_distance_matrix
qiime tools export --input-path $kmer_dir/bray_curtis_distance_matrix.qza --output-path $kmer_dir/bray_curtis_distance_matrix
qiime tools export --input-path $kmer_dir/jaccard_distance_matrix.qza --output-path $kmer_dir/jaccard_distance_matrix

Exported must_cm10000/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_cm10000/bray_curtis_distance_matrix
Exported must_cm10000/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_cm10000/jaccard_distance_matrix
Exported must_kmer_cm10000/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_kmer_cm10000/bray_curtis_distance_matrix
Exported must_kmer_cm10000/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_kmer_cm10000/jaccard_distance_matrix


In [10]:
%%bash
geodesic_path='/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/geodesic_distance_ITS/distance-matrix.tsv'

folder_path_1="must_cm10000"
folder_path_2="must_kmer_cm10000"

# Call the R script and pass folder paths and geodesic matrix
Rscript /home/lfloerl/microterroir/Microbiome/Other_scripts/mantel_test.R "$folder_path_1" "$folder_path_2" "$geodesic_path"

                                           Test Mantel_statistic_r Significance
1      must_cm10000-bray_curtis_distance_matrix         0.19513182        0.001
2          must_cm10000-jaccard_distance_matrix         0.17554377        0.001
3 must_kmer_cm10000-bray_curtis_distance_matrix         0.18935425        0.001
4     must_kmer_cm10000-jaccard_distance_matrix         0.05381511        0.019


In [11]:
mantel = pd.read_csv('must_kmer_cm10000/mantel_results.tsv', sep='\t')
mantel

,Test,Mantel_statistic_r,Significance
0,must_cm10000-bray_curtis_distance_matrix,0.195132,0.001
1,must_cm10000-jaccard_distance_matrix,0.175544,0.001
2,must_kmer_cm10000-bray_curtis_distance_matrix,0.189354,0.001
3,must_kmer_cm10000-jaccard_distance_matrix,0.053815,0.019


<hr>

# SUBSET TO SINGLE YEAR 

# Diversity 

In [103]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries/2021

In [12]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries/2021'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries/2021


In [73]:
%%bash 

# subset to only harvest samples before fitting 
qiime feature-table filter-samples \
  --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_filtered_table.qza \
  --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
  --p-where "[Collection_ID]='Harvest_2021'" \
  --o-filtered-table must_21_filtered_table.qza

qiime feature-table filter-seqs \
    --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
    --i-table must_21_filtered_table.qza \
    --o-filtered-data must_21_filtered_rep_seqs.qza

Saved FeatureTable[Frequency] to: must_21_filtered_table.qza
Saved FeatureData[Sequence] to: must_21_filtered_rep_seqs.qza


In [105]:
table = q2.Artifact.load('must_21_filtered_table.qza')

core_div_dict = {}
core_div = q2d.actions.core_metrics(table=table,
                                    sampling_depth=depth,
                                    metadata=md, 
                                    n_jobs=threads)
# save outputs
output_dir='must_21_cm{0}/'.format(depth)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for k, v in core_div.__dict__.items():
    if k != '_fields':
        v.save(output_dir + k)
        core_div_dict[k] = v

/scratch/lfloerl/.condaenvs/qiime2-amplicon-2024.10/lib/python3.10/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:146: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.18900461277761615 and the largest is 7.02508077461974.
  warn(


## Kmerizer 

In [106]:
%%bash

qiime kmerizer core-metrics \
    --i-sequences must_21_filtered_rep_seqs.qza \
    --i-table must_21_filtered_table.qza \
    --p-sampling-depth 10000 \
    --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
    --p-n-jobs 'auto' \
    --output-dir must_21_kmer_cm10000/

Saved FeatureTable[Frequency] to: must_21_kmer_cm10000/rarefied_table.qza
Saved FeatureTable[Frequency] to: must_21_kmer_cm10000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: must_21_kmer_cm10000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: must_21_kmer_cm10000/shannon_vector.qza
Saved DistanceMatrix to: must_21_kmer_cm10000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: must_21_kmer_cm10000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: must_21_kmer_cm10000/jaccard_pcoa_results.qza
Saved PCoAResults to: must_21_kmer_cm10000/bray_curtis_pcoa_results.qza
Saved Visualization to: must_21_kmer_cm10000/scatterplot.qzv


## Mantel 

In [107]:
%%bash 

div_dir='must_21_cm10000'
kmer_dir='must_21_kmer_cm10000'

# geodisic distance
#qiime tools export --input-path /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/geodesic_distance_ITS.qza --output-path geodesic_distance_ITS

# distance matrices 
qiime tools export --input-path $div_dir/bray_curtis_distance_matrix.qza --output-path $div_dir/bray_curtis_distance_matrix
qiime tools export --input-path $div_dir/jaccard_distance_matrix.qza --output-path $div_dir/jaccard_distance_matrix
qiime tools export --input-path $kmer_dir/bray_curtis_distance_matrix.qza --output-path $kmer_dir/bray_curtis_distance_matrix
qiime tools export --input-path $kmer_dir/jaccard_distance_matrix.qza --output-path $kmer_dir/jaccard_distance_matrix

Exported must_21_cm10000/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_21_cm10000/bray_curtis_distance_matrix
Exported must_21_cm10000/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_21_cm10000/jaccard_distance_matrix
Exported must_21_kmer_cm10000/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_21_kmer_cm10000/bray_curtis_distance_matrix
Exported must_21_kmer_cm10000/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory must_21_kmer_cm10000/jaccard_distance_matrix


In [13]:
%%bash
geodesic_path='/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/geodesic_distance_ITS/distance-matrix.tsv'

folder_path_1="must_21_cm10000"
folder_path_2="must_21_kmer_cm10000"

# Call the R script and pass folder paths and geodesic matrix
Rscript /home/lfloerl/microterroir/Microbiome/Other_scripts/mantel_test.R "$folder_path_1" "$folder_path_2" "$geodesic_path"

                                              Test Mantel_statistic_r
1      must_21_cm10000-bray_curtis_distance_matrix          0.2789328
2          must_21_cm10000-jaccard_distance_matrix          0.2124133
3 must_21_kmer_cm10000-bray_curtis_distance_matrix          0.2699840
4     must_21_kmer_cm10000-jaccard_distance_matrix          0.1017025
  Significance
1        0.001
2        0.001
3        0.001
4        0.002


In [14]:
mantel = pd.read_csv('must_21_kmer_cm10000/mantel_results.tsv', sep='\t')
mantel

,Test,Mantel_statistic_r,Significance
0,must_21_cm10000-bray_curtis_distance_matrix,0.278933,0.001
1,must_21_cm10000-jaccard_distance_matrix,0.212413,0.001
2,must_21_kmer_cm10000-bray_curtis_distance_matrix,0.269984,0.001
3,must_21_kmer_cm10000-jaccard_distance_matrix,0.101702,0.002


<hr>

# DENSE SAMPLING: SUBSET TO SINGLE PLOT IN HARVEST 2021  

-> calculate the Mantel test for every single densely sampled plot 

In [35]:
#!mkdir /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries/berries_2021_dense_sampling

In [16]:
workdir = '/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries/berries_2021_dense_sampling'
%cd $workdir

/home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/spatio-temporal/berries/berries_2021_dense_sampling


In [111]:
md_df = md.to_dataframe()
md_df['Plot_ID'].unique()

array(['Lavaux_Plot_4', 'Lavaux_Plot_5', 'Lavaux_Plot_9',
       'Lavaux_Plot_11', 'Lavaux_Plot_12', 'Lavaux_Plot_13',
       'Lavaux_Plot_14', 'Lavaux_Plot_15', 'Lavaux_Plot_18',
       'Lavaux_Plot_19', 'Lavaux_Plot_20', 'Lavaux_Plot_17'], dtype=object)

In [112]:
%%bash 

# Define the list of plots
plots=('Lavaux_Plot_4' 'Lavaux_Plot_5' 'Lavaux_Plot_9' 
       'Lavaux_Plot_11' 'Lavaux_Plot_12' 'Lavaux_Plot_13' 
       'Lavaux_Plot_14' 'Lavaux_Plot_15' 'Lavaux_Plot_18' 
       'Lavaux_Plot_19' 'Lavaux_Plot_20' 'Lavaux_Plot_17')

# Loop through each plot
for plot in "${plots[@]}"
do 
    echo "Processing $plot"
    
    # Create directory for the plot and navigate into it
    mkdir -p "./$plot"
    pushd "./$plot" > /dev/null

    # Filter table and rep seqs
    qiime feature-table filter-samples \
      --i-table /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/ITS/lavaux/must_harvest21_filtered_table.qza \
      --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux.tsv \
      --p-where "[Plot_ID]='$plot'" \
      --o-filtered-table must_harvest21_${plot}_filtered_table.qza

    qiime feature-table filter-seqs \
        --i-data /home/lfloerl/cloud/lfloerl/Microterroir/artifacts/Sven-denoising-test/ITS/revcomp-trimmed-denoised/with-adjusted-parameters/ITS-revcomp-trimmed-ER-0.05-denoised-trunc-190-PFA-4.0-maxee-4.0-rep-seqs.qza \
        --i-table must_harvest21_${plot}_filtered_table.qza \
        --o-filtered-data must_harvest21_${plot}_filtered_rep_seqs.qza

    # Core diversity metrics
    qiime diversity core-metrics \
        --i-table must_harvest21_${plot}_filtered_table.qza \
        --p-sampling-depth 10000 \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux_Harvest21.tsv \
        --p-n-jobs 10 \
        --output-dir cm10000/

    # Kmer diversity
    qiime kmerizer core-metrics \
        --i-sequences must_harvest21_${plot}_filtered_rep_seqs.qza \
        --i-table must_harvest21_${plot}_filtered_table.qza \
        --p-sampling-depth 10000 \
        --m-metadata-file /home/lfloerl/microterroir/Microbiome/Metadata/ITS_lavaux_Harvest21.tsv \
        --p-n-jobs auto \
        --output-dir kmer_cm10000/

    # Export all distance matrics for Mantel 
    qiime tools export --input-path cm10000/bray_curtis_distance_matrix.qza --output-path cm10000/bray_curtis_distance_matrix
    qiime tools export --input-path cm10000/jaccard_distance_matrix.qza --output-path cm10000/jaccard_distance_matrix
    qiime tools export --input-path kmer_cm10000/bray_curtis_distance_matrix.qza --output-path kmer_cm10000/bray_curtis_distance_matrix
    qiime tools export --input-path kmer_cm10000/jaccard_distance_matrix.qza --output-path kmer_cm10000/jaccard_distance_matrix

    # Return to the parent directory
    popd > /dev/null
done


Processing Lavaux_Plot_4
Saved FeatureTable[Frequency] to: must_harvest21_Lavaux_Plot_4_filtered_table.qza
Saved FeatureData[Sequence] to: must_harvest21_Lavaux_Plot_4_filtered_rep_seqs.qza
Saved FeatureTable[Frequency] to: cm10000/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: cm10000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: cm10000/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: cm10000/evenness_vector.qza
Saved DistanceMatrix to: cm10000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: cm10000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: cm10000/jaccard_pcoa_results.qza
Saved PCoAResults to: cm10000/bray_curtis_pcoa_results.qza
Saved Visualization to: cm10000/jaccard_emperor.qzv
Saved Visualization to: cm10000/bray_curtis_emperor.qzv
Saved FeatureTable[Frequency] to: kmer_cm10000/rarefied_table.qza
Saved FeatureTable[Frequency] to: kmer_cm10000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: kmer_cm10000/observed_features_ve

Saved DistanceMatrix to: cm10000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: cm10000/jaccard_pcoa_results.qza
Saved PCoAResults to: cm10000/bray_curtis_pcoa_results.qza
Saved Visualization to: cm10000/jaccard_emperor.qzv
Saved Visualization to: cm10000/bray_curtis_emperor.qzv
Saved FeatureTable[Frequency] to: kmer_cm10000/rarefied_table.qza
Saved FeatureTable[Frequency] to: kmer_cm10000/kmer_table.qza
Saved SampleData[AlphaDiversity] to: kmer_cm10000/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: kmer_cm10000/shannon_vector.qza
Saved DistanceMatrix to: kmer_cm10000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: kmer_cm10000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: kmer_cm10000/jaccard_pcoa_results.qza
Saved PCoAResults to: kmer_cm10000/bray_curtis_pcoa_results.qza
Saved Visualization to: kmer_cm10000/scatterplot.qzv
Exported cm10000/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory cm10000/bray_curtis_distance_

Saved DistanceMatrix to: kmer_cm10000/jaccard_distance_matrix.qza
Saved DistanceMatrix to: kmer_cm10000/bray_curtis_distance_matrix.qza
Saved PCoAResults to: kmer_cm10000/jaccard_pcoa_results.qza
Saved PCoAResults to: kmer_cm10000/bray_curtis_pcoa_results.qza
Saved Visualization to: kmer_cm10000/scatterplot.qzv
Exported cm10000/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory cm10000/bray_curtis_distance_matrix
Exported cm10000/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory cm10000/jaccard_distance_matrix
Exported kmer_cm10000/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory kmer_cm10000/bray_curtis_distance_matrix
Exported kmer_cm10000/jaccard_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory kmer_cm10000/jaccard_distance_matrix
Processing Lavaux_Plot_19
Saved FeatureTable[Frequency] to: must_harvest21_Lavaux_Plot_19_filtered_table.qza
Saved FeatureData[Sequence] to: must_harvest21_Lav

###  Mantel tests 

In [20]:
!ls geodesic_distance_ITS_HR/

distance-matrix.tsv


In [21]:
%%bash
folders=('Lavaux_Plot_4' 'Lavaux_Plot_5' 'Lavaux_Plot_9' 
       'Lavaux_Plot_11' 'Lavaux_Plot_12' 'Lavaux_Plot_13' 
       'Lavaux_Plot_14' 'Lavaux_Plot_15' 'Lavaux_Plot_18' 
       'Lavaux_Plot_19' 'Lavaux_Plot_20' 'Lavaux_Plot_17')

geodesic_path='geodesic_distance_ITS_HR/distance-matrix.tsv'

for folder in "${folders[@]}"
do
    folder_path_1="$folder/cm10000"
    folder_path_2="$folder/kmer_cm10000"

    # Call the R script and pass folder paths and geodesic matrix
    Rscript /home/lfloerl/microterroir/Microbiome/Other_scripts/mantel_test.R "$folder_path_1" "$folder_path_2" "$geodesic_path"
done

                                                    Test Mantel_statistic_r
1      Lavaux_Plot_4/cm10000-bray_curtis_distance_matrix        -0.01461738
2          Lavaux_Plot_4/cm10000-jaccard_distance_matrix        -0.03875348
3 Lavaux_Plot_4/kmer_cm10000-bray_curtis_distance_matrix        -0.01852599
4     Lavaux_Plot_4/kmer_cm10000-jaccard_distance_matrix        -0.08090353
  Significance
1        0.531
2        0.712
3        0.559
4        0.837
                                                    Test Mantel_statistic_r
1      Lavaux_Plot_5/cm10000-bray_curtis_distance_matrix         0.03276169
2          Lavaux_Plot_5/cm10000-jaccard_distance_matrix        -0.16853754
3 Lavaux_Plot_5/kmer_cm10000-bray_curtis_distance_matrix         0.07452180
4     Lavaux_Plot_5/kmer_cm10000-jaccard_distance_matrix        -0.24078233
  Significance
1        0.378
2        0.916
3        0.262
4        0.999
                                                    Test Mantel_statistic_r
1      Lavaux_

'nperm' >= set of all permutations: complete enumeration.
Set of permutations < 'minperm'. Generating entire set.
'nperm' >= set of all permutations: complete enumeration.
Set of permutations < 'minperm'. Generating entire set.
'nperm' >= set of all permutations: complete enumeration.
Set of permutations < 'minperm'. Generating entire set.
'nperm' >= set of all permutations: complete enumeration.
Set of permutations < 'minperm'. Generating entire set.


                                                     Test Mantel_statistic_r
1      Lavaux_Plot_19/cm10000-bray_curtis_distance_matrix          0.4857143
2          Lavaux_Plot_19/cm10000-jaccard_distance_matrix         -0.7142857
3 Lavaux_Plot_19/kmer_cm10000-bray_curtis_distance_matrix          0.4857143
4     Lavaux_Plot_19/kmer_cm10000-jaccard_distance_matrix          0.1428571
  Significance
1    0.1666667
2    1.0000000
3    0.1666667
4    0.5000000
                                                     Test Mantel_statistic_r
1      Lavaux_Plot_20/cm10000-bray_curtis_distance_matrix        0.035579552
2          Lavaux_Plot_20/cm10000-jaccard_distance_matrix        0.009441774
3 Lavaux_Plot_20/kmer_cm10000-bray_curtis_distance_matrix       -0.085800015
4     Lavaux_Plot_20/kmer_cm10000-jaccard_distance_matrix        0.137162277
  Significance
1        0.383
2        0.469
3        0.789
4        0.101
                                                     Test Mantel_statistic_r
1  

In [24]:
# List of folders to loop through
folders = ['Lavaux_Plot_4', 'Lavaux_Plot_5', 'Lavaux_Plot_9', 
           'Lavaux_Plot_11', 'Lavaux_Plot_12', 'Lavaux_Plot_13', 
           'Lavaux_Plot_14', 'Lavaux_Plot_15', 'Lavaux_Plot_18', 
           'Lavaux_Plot_19', 'Lavaux_Plot_20', 'Lavaux_Plot_17']

# Initialize an empty list to store the results
all_results = []

# Loop through each folder and process the results
for folder in folders:
    folder_path_2 = os.path.join(folder, "kmer_cm10000")
    mantel_path_2 = os.path.join(folder_path_2, "mantel_results.tsv")
    
    # Check if the mantel results file exists
    if os.path.exists(mantel_path_2):
        df_2 = pd.read_csv(mantel_path_2, sep="\t")

        # Add metadata columns
        df_2['Main_Folder'] = folder
        df_2['Subfolder'] = 'kmer_cm10000'
        df_2['Distance_Matrix'] = 'bray_curtis_distance_matrix' if 'bray_curtis' in mantel_path_2 else 'jaccard_distance_matrix'

        all_results.append(df_2)

# Concatenate all the results into a single dataframe
if all_results:
    final_results = pd.concat(all_results, ignore_index=True)
    final_results = final_results[['Main_Folder', 'Subfolder', 'Distance_Matrix', 'Mantel_statistic_r', 'Significance']]
else:
    final_results = pd.DataFrame(columns=['Main_Folder', 'Subfolder', 'Distance_Matrix', 'Mantel_statistic_r', 'Significance'])

# Display final results
final_results


,Main_Folder,Subfolder,Distance_Matrix,Mantel_statistic_r,Significance
0,Lavaux_Plot_4,kmer_cm10000,jaccard_distance_matrix,-0.014617,0.531000
1,Lavaux_Plot_4,kmer_cm10000,jaccard_distance_matrix,-0.038753,0.712000
2,Lavaux_Plot_4,kmer_cm10000,jaccard_distance_matrix,-0.018526,0.559000
3,Lavaux_Plot_4,kmer_cm10000,jaccard_distance_matrix,-0.080904,0.837000
4,Lavaux_Plot_5,kmer_cm10000,jaccard_distance_matrix,0.032762,0.378000
5,Lavaux_Plot_5,kmer_cm10000,jaccard_distance_matrix,-0.168538,0.916000
6,Lavaux_Plot_5,kmer_cm10000,jaccard_distance_matrix,0.074522,0.262000
7,Lavaux_Plot_5,kmer_cm10000,jaccard_distance_matrix,-0.240782,0.999000
8,Lavaux_Plot_9,kmer_cm10000,jaccard_distance_matrix,0.117361,0.101000
9,Lavaux_Plot_9,kmer_cm10000,jaccard_distance_matrix,0.029952,0.378000
